In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import coiled
import dask
import numpy as np
import xarray as xr
import xclim

from ocr.risks.fire import classify_wind_directions, direction_histogram, nws_fire_weather
from ocr.utils import load_conus404, prep_encoding

In [4]:
args = {
    'name': 'fire-weather-distribution',
    'region': 'us-west-2',
    'n_workers': [20, 40],
    'tags': {'Project': 'OCR'},
    'worker_vm_types': 'm8g.2xlarge',
    'scheduler_vm_types': 'm8g.2xlarge',
}
cluster = coiled.Cluster(**args)
client = cluster.get_client()

Output()

╭──────────────────────────────── Package Info ────────────────────────────────╮
│           ╷                                                                  │
│   Package │ Note                                                             │
│ ╶─────────┼────────────────────────────────────────────────────────────────╴ │
│   ocr     │ Wheel built from                                                 │
│           │ /opt/coiled/env/.path-deps/ocr-0.0.post89+g1997206-py3-none-an   │
│           │ y.whl                                                            │
│           ╵                                                                  │
╰──────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────── Not Synced with Cluster ───────────────────────────╮
│                                ╷                                 ╷           │
│   Package                      │ Error                           │ Level     │
│ ╶──────────────────────────────┼─────────────────────────────────┼─────────╴ │
│   s5cmd                        │ s5cmd~=2.3.0 has no install     │ Warning   │
│                                │ candidate for Python 3.13       │           │
│                                │ linux-aarch64 on conda-forge    │           │
│   libopenvino-intel-npu-plugin │ libopenvino-intel-npu-plugin~=… │ Warning   │
│                                │ has no install candidate for    │           │
│                                │ Python 3.13 linux-aarch64 on    │           │
│                                │ conda-forge                     │           │
│   libopenvino-intel-gpu-plugin │ libopenvino-intel-gpu-plugin~=… │ Warning   │
│                                │ has no install candidate for    │           │
│                                │ Python 3.13 linux-aarch64 on    │           │
│                                │ conda-forge                     │           │
│   libopenvino-intel-cpu-plugin │ libopenvino-intel-cpu-plugin~=… │ Warning   │
│                                │ has no install candidate for    │           │
│                                │ Python 3.13 linux-aarch64 on    │           │
│                                │ conda-forge                     │           │
│   libcxxabi                    │ libcxxabi~=20.1.8 has no        │ Warning   │
│                                │ install candidate for Python    │           │
│                                │ 3.13 linux-aarch64 on           │           │
│                                │ conda-forge                     │           │
│   level-zero                   │ level-zero~=1.24.0 has no       │ Warning   │
│                                │ install candidate for Python    │           │
│                                │ 3.13 linux-aarch64 on           │           │
│                                │ conda-forge                     │           │
│                                ╵                                 ╵           │
╰──────────────────────────────────────────────────────────────────────────────╯

Output()

2025-08-05 06:13:37,409 - distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=20 maximum=40
Task exception was never retrieved
future: <Task finished name='Task-3469' coro=<Client._gather.<locals>.wait() done, defined at /opt/coiled/env/lib/python3.13/site-packages/distributed/client.py:2385> exception=AllExit()>
Traceback (most recent call last):
  File "/opt/coiled/env/lib/python3.13/site-packages/distributed/client.py", line 2394, in wait
    raise AllExit()
distributed.client.AllExit


In [6]:
# Make relative humidity intermediate data variable

In [7]:
%%time
recreate_rh = True
if recreate_rh:
    ds = load_conus404('T2')
    ds['TD2'] = load_conus404('TD2')['TD2']
    hurs = xclim.indicators.atmos.relative_humidity_from_dewpoint(tas=ds['T2'], tdps=ds['TD2'])
    hurs = dask.optimize(hurs)[0]
    hurs.to_zarr(
        's3://carbonplan-scratch/intermediate/met-data/conus404/hurs.zarr',
        consolidated=True,
        mode='w',
    )

/opt/coiled/env/lib/python3.13/site-packages/xclim/core/cfchecks.py:79: UserWarning: Variable does not have a `standard_name` attribute.
  check_valid(vardata, "standard_name", data["standard_name"])
/opt/coiled/env/lib/python3.13/site-packages/zarr/api/asynchronous.py:228: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


CPU times: user 10.5 s, sys: 276 ms, total: 10.8 s
Wall time: 5min 21s


In [8]:
# Make sfcWindspeed and sfcWindfromdir intermediate variables

In [9]:
%%time
recreate_winds = True
if recreate_winds:
    ds = load_conus404('U10')
    ds['V10'] = load_conus404('V10')['V10']
    winds = xclim.indicators.atmos.wind_speed_from_vector(uas=ds['U10'], vas=ds['V10'])[
        0
    ].to_dataset()
    winds['sfcWindfromdir'] = xclim.indicators.atmos.wind_speed_from_vector(
        uas=ds['U10'], vas=ds['V10']
    )[1]
    winds = dask.optimize(winds)[0]
    winds.to_zarr(
        's3://carbonplan-scratch/intermediate/met-data/conus404/winds.zarr',
        consolidated=True,
        mode='w',
    )

/opt/coiled/env/lib/python3.13/site-packages/xclim/core/cfchecks.py:79: UserWarning: Variable does not have a `standard_name` attribute.
  check_valid(vardata, "standard_name", data["standard_name"])
/opt/coiled/env/lib/python3.13/site-packages/xclim/core/cfchecks.py:79: UserWarning: Variable does not have a `standard_name` attribute.
  check_valid(vardata, "standard_name", data["standard_name"])
/opt/coiled/env/lib/python3.13/site-packages/zarr/api/asynchronous.py:228: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
/opt/coiled/env/lib/python3.13/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 11.57 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/

CPU times: user 7.14 s, sys: 181 ms, total: 7.32 s
Wall time: 6min 45s


In [ ]:
# calculate whether there is fire weather

In [10]:
hurs = xr.open_zarr('s3://carbonplan-scratch/intermediate/met-data/conus404/hurs.zarr')
wind = xr.open_zarr('s3://carbonplan-scratch/intermediate/met-data/conus404/winds.zarr')
fire_weather_mask = nws_fire_weather(
    hurs['hurs'],
    15,
    # reason that wind gusts are typically ~40% higher than average wind speed
    # and we want to base this on wind gusts (need a citation for this)
    wind['sfcWind'] * 1.4,
    35,
)

In [11]:
fire_weather_mask = dask.optimize(fire_weather_mask)[0]

In [12]:
%%time
# classify the winds into 8 cardinal directions
direction_indices = classify_wind_directions(wind['sfcWindfromdir'].where(fire_weather_mask))

CPU times: user 84.1 ms, sys: 14.1 ms, total: 98.2 ms
Wall time: 14.4 s


In [13]:
%%time
# create the distribution of winds based upon the classified wind directions
# if there are nans in the mix, they won't contribute to the distribution
fraction = xr.apply_ufunc(
    direction_histogram,
    direction_indices,
    input_core_dims=[['time']],
    output_core_dims=[['wind_direction']],
    output_sizes={'wind_direction': 8},
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float],
    kwargs={},
)

CPU times: user 10.3 ms, sys: 6 μs, total: 10.3 ms
Wall time: 10.2 ms


<timed exec>:3: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.


In [14]:
fraction = fraction.assign_coords(wind_direction=np.arange(0, 8))  # .chunk({'x': 500, 'y': 500})
fraction = dask.optimize(fraction)[0]

In [15]:
encoding = prep_encoding(fraction.to_dataset())

In [15]:
%%time
fraction.to_zarr(
    's3://carbonplan-scratch/intermediate/met-data/conus404/fire_weather_wind_distribution.zarr',
    zarr_format=3,
    encoding=encoding,
    mode='w',
    consolidated=True,
)

/opt/coiled/env/lib/python3.13/site-packages/zarr/core/group.py:2552: UserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(
/opt/coiled/env/lib/python3.13/site-packages/zarr/api/asynchronous.py:228: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
/opt/coiled/env/lib/python3.13/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 14.77 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


CPU times: user 2.42 s, sys: 137 ms, total: 2.55 s
Wall time: 6min 3s


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7a41bbdbf4d0>>
Traceback (most recent call last):
  File "/opt/coiled/env/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

KeyboardInterrupt


KeyboardInterrupt



In [ ]:
%%time
fraction = xr.open_zarr(
    's3://carbonplan-scratch/intermediate/met-data/conus404/fire_weather_wind_distribution.zarr'
)
# sum up the fractions of all of the wind directions. if any pixel sums to zero it will
# fall into the no_fire_weather mask and be cast to a `nan`. if any pixel has any fraction
# of timestamps with fire weather then it will be cast into the mask where there *is* fire weather
no_fire_weather = fraction.sum(dim='wind_direction')
mode = fraction.argmax(dim='wind_direction').where(no_fire_weather).chunk({'x': -1, 'y': -1})

encoding = prep_encoding(mode.to_dataset())

mode.to_zarr(
    's3://carbonplan-scratch/intermediate/met-data/conus404/fire_weather_wind_mode.zarr',
    zarr_format=3,
    encoding=encoding,
    mode='w',
    consolidated=True,
)

In [ ]:
test = xr.open_zarr(
    's3://carbonplan-scratch/intermediate/met-data/conus404/fire_weather_wind_mode.zarr'
)

In [ ]:
test

In [16]:
cluster.close()

2025-08-05 06:36:08,056 - distributed.deploy.adaptive - INFO - Adaptive scaling stopped: minimum=20 maximum=40. Reason: unknown


ERROR! Session/line number was not unique in database. History logging moved to new session 5
